In [15]:
import numpy as np
import cvxpy as opt
import pandas as pd
import csv
import pprint
import glob

In [16]:
"""
This cell will develop the time constraint matrix. We define this matrix as a 168 * 5 (i, j) 
matrix where i represent the 5 minute block of time and 
In this 1 and 0 matrix, 1 represents a scheduled class and 0 implies an empty spot. 
We can set a linear constraint in our linear program that says all must be less than 2. This 
constraint will be later in our optimization. 
"""
time_matrix = np.zeros((168,5))
time_constraint = (time_matrix <= 1)



In [23]:
"""
In this cell we will import all of the csv files and turn them into a parsable dataframe.
We will need to iterate through all of the files in data subfolder. 
"""
# Need to make a list of all file names
course_files = glob.glob('data/*.csv')

def pull_names(course_files):
    """
    Takes in a set of course files that have format {'data/*.csv'} and outputs a set of just department names
    """
    names = [] 
    for file in course_files:
        #take out the data
        pre, rest = file.split("/")
        #take out CSV
        index_name, blah = rest.split('.')
        #take out quarter
        department, quarter = index_name.split("-")
        names.append(department)
    return names

course_file_names = pull_names(course_files)
print("The following datasets have been properly uploaded " + str(course_file_names))

#Now we need to turn these bad boys into data frames. 
def create_dataframes(names_list, quarter):
    """
    Takes a set of CSV names (as a list) and outputs a dictionary of pandas frames 
    that can be easily accessed by department. 
    """
    pandas_frames = {}
    for file_name in names_list:
        pandas_frames[file_name] = pd.read_csv("data/" + file_name + "-" + quarter + ".csv")
        pandas_frames[file_name] = pandas_frames[file_name].reindex(index=pandas_frames[file_name].index[::-1]).reset_index()
    return pandas_frames

quarter = "WINTER2019"
frames = create_dataframes(course_file_names, quarter)

The following datasets have been properly uploaded ['MATH', 'PSTAT', 'CMPSC']


In [ ]:
frames["CMP"]

In [ ]:
#Optimization Solver Cell
cons = [time_constraint]
